In [2]:
consumer_key = 'twKlZH3suYkRJRYBnJFGLCCnL'
consumer_secret = 'RBgW7DfW0FxcRl3VXm76tBJ02LmPVdQL8A5RoeMQrAF5lW61bq'
access_token = '107991822-bjjdRQGUGSRLMtQVnOnSPY8yqLOVrf81uWCMvHKy'
access_token_secret = 'V2lRDvXjveAh26FLhghMOt6aY0xWS9fARfDOCVOCs6sPg'

In [5]:
from tweepy import API
from tweepy.auth import OAuthHandler
from tweepy import Cursor
import json
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = API(auth)

WORLD_WIDE_ID = 1
US_ID = 23424977

world_trends = api.trends_place(WORLD_WIDE_ID)
print(json.dumps(world_trends,indent=1))

us_trends = api.trends_place(US_ID)
print(json.dumps(us_trends,indent=1))

[
 {
  "locations": [
   {
    "woeid": 1,
    "name": "Worldwide"
   }
  ],
  "trends": [
   {
    "tweet_volume": 468615,
    "promoted_content": null,
    "url": "http://twitter.com/search?q=%23R3peatTheGloryAteneo",
    "query": "%23R3peatTheGloryAteneo",
    "name": "#R3peatTheGloryAteneo"
   },
   {
    "tweet_volume": null,
    "promoted_content": null,
    "url": "http://twitter.com/search?q=%23ALLINLaSalle",
    "query": "%23ALLINLaSalle",
    "name": "#ALLINLaSalle"
   },
   {
    "tweet_volume": 43754,
    "promoted_content": null,
    "url": "http://twitter.com/search?q=%23RIPJustinHair",
    "query": "%23RIPJustinHair",
    "name": "#RIPJustinHair"
   },
   {
    "tweet_volume": 16204,
    "promoted_content": null,
    "url": "http://twitter.com/search?q=%23SarayaDirenDavuto%C4%9Flu",
    "query": "%23SarayaDirenDavuto%C4%9Flu",
    "name": "#SarayaDirenDavuto\u011flu"
   },
   {
    "tweet_volume": 207697,
    "promoted_content": null,
    "url": "http://twitter.com/searc

In [ ]:
import json

with open('worldTrends.json','w') as outfile:
    json.dump(world_trends,outfile)
with open('usTrends.json','w') as outfile:
    json.dump(us_trends,outfile)

In [6]:
world_trends_set = set([trend['name'] for trend in world_trends[0]['trends']])
us_trends_set = set([trend['name'] for trend in us_trends[0]['trends']]) 

common_trends = world_trends_set.intersection(us_trends_set)
print(common_trends)

{'#ALDUBHBDBossing', '#YTFFManila2016', '#TopMentirasParaTerminar', '#N1CanalFiesta18', '#HurumaTragedy', '#RazonesParaTomar', '#AFLGiantsHawks', 'Austin Rivers', '#PeopleAssumeIAm', '#RIPJustinHair', '#ALLINLaSalle'}


In [ ]:
import csv,
    
    # we are trying to analyze the hash Tag NotGreatTVshows which is currently trending.
    csvFile = open('not-great-tv-shows-full.csv', 'a')
    #Use csv Writer
    csvWriter = csv.writer(csvFile, lineterminator='\n')
    csvWriter.writerow(['id','created_at','text','retweet_count','favorite_count','retweeted','entities','coordinates','user'])
    for tweet in Cursor(api.search, q='#NotGreatTVShows', lang='en', count='100000').items() :
            try:
               csvWriter.writerow([tweet.id_str,tweet.created_at, tweet.text.encode('utf-8'),
                                   tweet.retweet_count,tweet.favorite_count,
                                   tweet.entities,tweet.coordinates,tweet.user])
            except:
                continue

In [132]:
    import nltk
    import csv
    import string
    import re
    from nltk.util import ngrams
    def untokenize(ngram):
       tokens = list(ngram)
       return "".join([" "+i if not i.startswith("'") and \
                             i not in string.punctuation and \
                             i != "n't"
                          else i for i in tokens]).strip()
    non_speaker = re.compile('[A-Za-z]+: (.*)')

    def extract_phrases(text, phrase_counter, length):
        for sent in nltk.sent_tokenize(text):
            strip_speaker = non_speaker.match(sent)
            if strip_speaker is not None:
                sent = strip_speaker.group(1)
            words = nltk.word_tokenize(sent)
            for phrase in ngrams(words, length):
                if all(word not in string.punctuation for word in phrase):
                    phrase_counter[phrase] += 1
     

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/akhilch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [133]:
import pandas as pd
from collections import Counter

tweets = pd.read_csv('not-great-tv-shows-full.csv')
tweets.drop_duplicates(subset = 'id')

phrase_counter = Counter()

for index,tweet in tweets.iterrows():
    tweet_text = re.sub(r"http\S+", "", tweet['text']) 
    tweet_text = re.sub(r"x[a-f0-9][a-f0-9]","", tweet_text)
    tweet_text = re.sub(r"\\n","", tweet_text)
    tweet_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet_text)
                          .split())
    tweet_text = re.sub(r"^'|'$", '', tweet_text)
    tweet_text = tweet_text.strip()
    tweet_text = re.sub(r"^b RT Celebrity|^b RT |^b", "", tweet_text)
    tweet_text = re.sub(r" n*$","", tweet_text) 
    tweet_text = tweet_text.strip()
    extract_phrases(tweet_text,phrase_counter,4)
most_common_phrases = phrase_counter.most_common(50)
for ngrams,count in most_common_phrases:
    print('{0:<5}'.format(count),ngrams)

108   ('You', 'Think', 'You', 'Can')
98    ('So', 'You', 'Think', 'You')
82    ('Orange', 'is', 'the', 'new')
79    ('is', 'the', 'new', 'black')
76    ('presents', 'Orange', 'is', 'the')
76    ('Fox', 'presents', 'Orange', 'is')
59    ('Think', 'You', 'Can', 'Scrapbook')
57    ('ck14', 'So', 'You', 'Think')
55    ('M', 'U', 'S', 'H')
51    ('Who', 's', 'the', 'Bossk')
49    ('How', 'I', 'Met', 'Your')
44    ('M', 'A', 'T', 'H')
43    ('t', 'Call', 'Me', 'Shirley')
43    ('Laverne', 'Don', 't', 'Call')
43    ('Don', 't', 'Call', 'Me')
42    ('Dancing', 'With', 'The', 'Czars')
41    ('lives', 'Laverne', 'Don', 't')
40    ('Two', 'and', 'a', 'Half')
40    ('ck14', 'M', 'A', 'T')
39    ('N', 'C', 'L', 'E')
39    ('U', 'N', 'C', 'L')
37    ('Little', 'Outhouse', 'On', 'The')
37    ('Outhouse', 'On', 'The', 'Prairie')
36    ('Dream', 'Of', 'Diaper', 'Genies')
36    ('I', 'Dream', 'Of', 'Diaper')
35    ('Super', 'friends', 'with', 'benefits')
31    ('ck14', 'Everybody', 'Loves', 'Ramen')
31 

In [168]:
tv_shows = pd.read_csv('tvshows.csv')
tv_shows

,name,junk
0,NBC Nightly News,1970–present)
1,i dream of jeannie,19670)
2,On the Money,1970–present)
3,The Man from U N C L E,1974-present)
4,Great Performances,1972–present)
5,The Price Is Right,1972–present)
6,The Young and the Restless,1973–present)
7,Nova,1974–present)
8,Good Morning America,1975–present)
9,PBS NewsHour,1975–present)


In [169]:
from difflib import SequenceMatcher
from collections import Counter
#The SequenceMatcher class compares two sequences of any types, as long as the values are hashable. 
#It uses an algorithm to identify the longest contiguous matching blocks from the sequences, 
#eliminating “junk” values that do not contribute to the real data.
notGreatTvShowCounter = Counter()
for ngrams,count in most_common_phrases:
    showName = untokenize(ngrams)
    found = False
    for index,row in tv_shows.iterrows():
        if(SequenceMatcher(None, showName.lower(), row['name'].lower()).ratio() > 0.65):
            notGreatTvShowCounter[row['name']] += count
            print(showName," - ",row['name'])
            found = True
    if found == False:
        print(showName," - Not Found")

You Think You Can  -  So You Think You Can Dance
So You Think You  -  So You Think You Can Dance
Orange is the new  -  Orange Is the New Black
is the new black  -  Orange Is the New Black
presents Orange is the  - Not Found
Fox presents Orange is  - Not Found
Think You Can Scrapbook  - Not Found
ck14 So You Think  - Not Found
M U S H  -  M U S H
Who s the Bossk  - Not Found
How I Met Your  -  How I Met Your Mother
M A T H  -  M U S H
t Call Me Shirley  -  Don't Call Me Shirley
Laverne Don t Call  - Not Found
Don t Call Me  -  Don't Call Me Shirley
Dancing With The Czars  -  Dancing with the Stars
lives Laverne Don t  - Not Found
Two and a Half  -  Two and a Half Men
ck14 M A T  - Not Found
N C L E  -  Uncle
U N C L  -  Uncle
Little Outhouse On The  - Not Found
Outhouse On The Prairie  - Not Found
Dream Of Diaper Genies  - Not Found
I Dream Of Diaper  -  i dream of jeannie
Super friends with benefits  -  Friends with Benefits
ck14 Everybody Loves Ramen  - Not Found
Bob s Vegan Burgers  

In [170]:
for show,count in notGreatTvShowCounter.most_common(50):
    print('{0:<5}'.format(notGreatTvShowCounter.get(show)),show)

206   So You Think You Can Dance
161   Orange Is the New Black
126   M U S H
86    Don't Call Me Shirley
78    Uncle
58    Whose Line Is It Anyway
50    Good Morning America
49    How I Met Your Mother
42    Dancing with the Stars
40    Two and a Half Men
36    i dream of jeannie
35    Friends with Benefits
31    Bob's Burgers
30    The Real Housewives of Atlanta
30    The Real Housewives of Potomac
30    The Real Housewives of New Jersey
29    America's Got Talent
28    It's Always Sunny in Philadelphia
26    Keeping Up with the Kardashians
25    Catch a Contractor
